## --- Day 25: Sea Cucumber ---
Find somewhere safe to land your submarine. What is the first step on which no sea cucumbers move?

In [186]:
from aoc_utils import str_to_2d_array, print_grid
from itertools import product

STEP_END = "<<step boundary>>"

class CukeSim:
    def __init__(self, map_str):
        self.cukemap = str_to_2d_array(map_str)
        self.map_rows, self.map_cols = len(self.cukemap), len(self.cukemap[0])
        self.swapq = []
        self.steps = 0

        # Initial pass: iterate the entire map and find coords where a swap will occur on the first step
        for r, c in product(range(self.map_rows), range(self.map_cols)):
            self._scan_point(r, c)
        self.swapq.append(STEP_END)
        
    @classmethod
    def from_file(cls, filename):
        with open(filename) as f:
            in_str = f.read()

        return CukeSim(in_str)

    def _scan_point(self, r, c):
        """Scans the point at r,c to see which swaps need to be made in the next step (if any)"""

        # Add swap(s) only if space r,c is currently empty
        if self.cukemap[r][c] == ".":
            # Swap #1: r,c is empty and there's a > directly to the left
            if self.cukemap[r][c-1] == ">":          
                # Swap #1a: there's a v up-left waiting to occupy the space being opened by this > move
                if self.cukemap[r-1][c-1] == "v":
                    self.swapq.append(((r, c), (r-1, c-1)))    # swap the empty space up-left
                # Swap #1b: there's no v up-left
                else:
                    self.swapq.append(((r, c), (r, c-1)))    # swap the empty space left
            # Swap #2: r,c is empty, there's a v directly above, (and no > to the left)
            elif self.cukemap[r-1][c] == "v":
                self.swapq.append(((r,c), (r-1, c)))     # swap the empty space up

    def _swap(self, oldr, oldc, newr, newc):
        self.cukemap[oldr][oldc], self.cukemap[newr][newc] = self.cukemap[newr][newc], self.cukemap[oldr][oldc]

    def _do_step(self):
        curr_swap = self.swapq.pop(0)
        scan_points = set()

        def scan_point(r, c):
            """translate indices that cross edges"""
            scan_r = r % self.map_rows if r >= 0 else self.map_rows + r
            scan_c = c % self.map_cols if c >= 0 else self.map_cols + c
            scan_points.add((scan_r, scan_c))

        while curr_swap != STEP_END:
            # do all swaps in this step
            (oldr, oldc), (newr, newc) = curr_swap

            # always scan the new position of the empty space
            scan_point(newr, newc)

            if oldr == newr:        # left-right swap, then scan right of the new > loc by 1
                self._swap(oldr, oldc, newr, newc)
                scan_point(oldr, (oldc + 1) % self.map_cols)
            elif oldc == newc:      # up-down swap, then scan down of the new v loc by 1
                self._swap(oldr, oldc, newr, newc)
                scan_point((oldr + 1) % self.map_rows, oldc)
            else:   # special case - left-right >/. swap THEN left-upleft ./v swap
                self.cukemap[oldr][oldc] = ">"
                self.cukemap[oldr][oldc-1] = "v"
                self.cukemap[newr][newc] = "."
                # scan to the right of > and down from v
                scan_point(oldr, (oldc + 1) % self.map_cols)
                scan_point((oldr + 1) % self.map_rows, oldc - 1)

            curr_swap = self.swapq.pop(0)

        # Finally, scan all updated points to find swaps for the next step
        for r, c in scan_points:
            self._scan_point(r, c)

        self.swapq.append(STEP_END)
        self.steps += 1

    def run(self):
        """run until there are no more moves to make, return the first step w/o movement"""
        while len(self.swapq) > 1:
            self._do_step()
            if self.steps % 5 == 0:
                print(f"Step {self.steps}, q size {len(self.swapq)}")

        return self.steps + 1
    
    def print_grid(self):
        print_grid(self.cukemap, space_str="")


In [187]:
test = """
..........
.>v....v..
.......>..
..........
"""

test1 = """
..........
.>........
..........
..........
"""

test2 = """
...>...
.......
......>
v.....>
......>
.......
..vvv..
"""

testsim = CukeSim(test2)
testsim.print_grid()

...>...
.......
......>
v.....>
......>
.......
..vvv..


In [188]:
ex1_input = """
v...>>.vv>
.vv>>.vv..
>>.>v>...v
>>v>>.>.v.
v>v.vv.v..
>.>>..v...
.vv..>.>v.
v.v..>>v.v
....v..v.>
"""

ex1_sim = CukeSim(ex1_input)
assert 58 == ex1_sim.run()

Step 5, q size 18
Step 10, q size 14
Step 15, q size 16
Step 20, q size 9
Step 25, q size 12
Step 30, q size 13
Step 35, q size 14
Step 40, q size 7
Step 45, q size 9
Step 50, q size 9
Step 55, q size 3


In [189]:
p1_input = "./inputs/Day25.txt"
p1_sim = CukeSim.from_file(p1_input)
p1_sim.run()

Step 5, q size 3647
Step 10, q size 3419
Step 15, q size 3205
Step 20, q size 3174
Step 25, q size 2887
Step 30, q size 2805
Step 35, q size 2600
Step 40, q size 2468
Step 45, q size 2419
Step 50, q size 2344
Step 55, q size 2300
Step 60, q size 2213
Step 65, q size 2260
Step 70, q size 2220
Step 75, q size 2150
Step 80, q size 2056
Step 85, q size 2134
Step 90, q size 2096
Step 95, q size 2055
Step 100, q size 2071
Step 105, q size 2054
Step 110, q size 1939
Step 115, q size 1772
Step 120, q size 1607
Step 125, q size 1576
Step 130, q size 1543
Step 135, q size 1526
Step 140, q size 1495
Step 145, q size 1476
Step 150, q size 1494
Step 155, q size 1499
Step 160, q size 1462
Step 165, q size 1332
Step 170, q size 1267
Step 175, q size 1235
Step 180, q size 1207
Step 185, q size 1170
Step 190, q size 1052
Step 195, q size 977
Step 200, q size 912
Step 205, q size 868
Step 210, q size 850
Step 215, q size 860
Step 220, q size 906
Step 225, q size 945
Step 230, q size 954
Step 235, q size

498